In [1]:
# for data
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime, timedelta
from pyNBA.Data.data import QueryData
from pyNBA.Models.helpers import CleanData
import math

# for features
from pyNBA.Models.features import FeatureCreation
from pyNBA.Models.cluster import Cluster, Evaluate
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from research import Helpers
from statsmodels.graphics.api import abline_plot

# for statistical tests
from scipy.stats import shapiro
import pingouin as pg

# for machine learning
from sklearn import model_selection, preprocessing, ensemble, neighbors, linear_model, svm, neural_network, metrics
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# for explainer
from lime import lime_tabular

# misc
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/Users/brandonshimiaie/Projects/pyNBA/venv/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.3, the latest is 0.3.8.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
import pandas as pd
import numpy as np
import time
from nba_api.stats.endpoints import SynergyPlayTypes
from pyNBA.Models.constants import PLAY_TYPES
from pyNBA.Models.features import FeatureCreation
from pyNBA.Data.helpers import Helpers

In [3]:
from pyNBA.Props.player_props import PlayerProps

In [4]:
pp = PlayerProps()

current_date_string = datetime.now().strftime("%Y-%m-%d")

historical_boxscores = pp.get_historical_boxscores()
historical_boxscores['NAME'] = None
historical_boxscores['POSITION'] = None
        
currentplayers = pp.get_current_players()

boxscores = historical_boxscores.append(currentplayers)

relevant_seasons = ['2020-21']
boxscores = boxscores.loc[boxscores['SEASON'].isin(relevant_seasons)]

/Users/brandonshimiaie/Projects/pyNBA/sqlite/db/nba.db
2.6.0


In [5]:
feature_creation = FeatureCreation()
helpers = Helpers()

In [6]:
boxscores['ATTEMPTS'] = boxscores['TOTAL_ATTEMPTS']
boxscores['ATTEMPTS/POSSESSION'] = boxscores['ATTEMPTS']/boxscores['POSS']

# average attempts per possession
boxscores = feature_creation.expanding_weighted_mean(
    df=boxscores, group_col_names=['SEASON', 'TEAM', 'PLAYERID'], col_name='ATTEMPTS/POSSESSION',
    new_col_name='AVG_ATTEMPTS/POSSESSION', weight_col_name='POSS'
)

boxscores['POINTS/ATTEMPT'] = boxscores['PTS']/boxscores['ATTEMPTS']

# average points per attempt
boxscores = feature_creation.expanding_weighted_mean(
    df=boxscores, group_col_names=['SEASON', 'TEAM', 'PLAYERID'], col_name='POINTS/ATTEMPT',
    new_col_name='AVG_POINTS/ATTEMPT', weight_col_name='ATTEMPTS'
)

# adjustment for defense (points per attempt)
for play_type in PLAY_TYPES:
    player_play_type_data = pd.DataFrame()
    team_play_type_data = pd.DataFrame()
    for season in relevant_seasons:
        player_data = helpers.get_play_type_breakdown(play_type, season, 'player')

        player_data['SEASON'] = season
        player_data['PLAYER_ID'] = player_data['PLAYER_ID'].apply(lambda x: str(x))
        player_data = player_data.rename(columns={
            'TEAM_ABBREVIATION': 'TEAM', 'PLAYER_ID': 'PLAYERID',
            'PPP': '{}_PPP'.format(play_type), 'POSS_PCT': '{}_POSS_PCT'.format(play_type)
            })
        player_data = player_data[
            ['SEASON', 'PLAYERID', 'TEAM', '{}_PPP'.format(play_type), '{}_POSS_PCT'.format(play_type)]
            ]

        player_play_type_data = player_play_type_data.append(player_data)

        team_data = helpers.get_play_type_breakdown(play_type, season, 'team')

        team_data['SEASON'] = season
        team_data = team_data.rename(columns={
            'TEAM_ABBREVIATION': 'OPP_TEAM', 'PPP': '{}_PPP_ALLOWED'.format(play_type),
            'POSS_PCT': '{}_POSS_PCT_ALLOWED'.format(play_type)
            })
        team_data = team_data[
            ['SEASON', 'OPP_TEAM', '{}_PPP_ALLOWED'.format(play_type), '{}_POSS_PCT_ALLOWED'.format(play_type)]
            ]

        team_play_type_data = team_play_type_data.append(team_data)

    boxscores = boxscores.merge(player_play_type_data, on=['SEASON', 'PLAYERID', 'TEAM'], how='left')
    boxscores = boxscores.merge(team_play_type_data, on=['SEASON', 'OPP_TEAM'], how='left')

poss_pct_cols = ['{}_POSS_PCT'.format(i) for i in PLAY_TYPES]
poss_pct_allowed_cols = ['{}_POSS_PCT_ALLOWED'.format(i) for i in PLAY_TYPES]
boxscores[poss_pct_cols] = boxscores[poss_pct_cols].replace([0], 0.001)
boxscores[poss_pct_allowed_cols] = boxscores[poss_pct_allowed_cols].replace([0], 0.001)
boxscores['TOTAL_POSS_PCT'] = boxscores[poss_pct_cols].sum(axis=1)
boxscores['TOTAL_POSS_PCT_ALLOWED'] = boxscores[poss_pct_allowed_cols].sum(axis=1)
boxscores[poss_pct_cols] = boxscores[poss_pct_cols].div(boxscores['TOTAL_POSS_PCT'], axis=0)
boxscores[poss_pct_allowed_cols] = boxscores[poss_pct_allowed_cols].div(boxscores['TOTAL_POSS_PCT_ALLOWED'], axis=0)

boxscores['NET_POINTS/ATTEMPT'] = 0
boxscores['TOTAL_POSS_PCT'] = 0
boxscores['IMPLIED_NET_POINTS/ATTEMPT'] = 0
boxscores['TOTAL_IMPLIED_POSS_PCT'] = 0

In [7]:
display(boxscores)

,GAMEID,PLAYERID,TEAM,DATE,OPP_TEAM,COMMENT,START,SECONDSPLAYED,PTS,FGM,FGA,FG3M,FG3A,FTM,FTA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,PCT_AST_2PM,PCT_AST_3PM,OREB,OREB_PCT,DREB,DREB_PCT,AST,AST_PCT,AST_RATIO,STL,BLK,TOV,PF,PLUSMINUS,USG_PCT,PACE,POSS,TOTAL_ATTEMPTS,TOTAL_PTS,TOTAL_FTA,TOTAL_FTM,SHOT_ATTEMPTS,SHOT_PTS,SHOT_FTA,SHOT_FTM,SFOUL_ATTEMPTS,SFOUL_PTS,SFOUL_FTA,SFOUL_FTM,PFOUL_ATTEMPTS,PFOUL_PTS,PFOUL_FTA,PFOUL_FTM,TFOUL_ATTEMPTS,TFOUL_PTS,TFOUL_FTA,TFOUL_FTM,ID,SEASON,SEASONTYPE,HTM,VTM,W,NAME,POSITION,ATTEMPTS,ATTEMPTS/POSSESSION,AVG_ATTEMPTS/POSSESSION,POINTS/ATTEMPT,AVG_POINTS/ATTEMPT,Transition_PPP,Transition_POSS_PCT,Transition_PPP_ALLOWED,Transition_POSS_PCT_ALLOWED,PRBallHandler_PPP,PRBallHandler_POSS_PCT,PRBallHandler_PPP_ALLOWED,PRBallHandler_POSS_PCT_ALLOWED,PRRollman_PPP,PRRollman_POSS_PCT,PRRollman_PPP_ALLOWED,PRRollman_POSS_PCT_ALLOWED,Postup_PPP,Postup_POSS_PCT,Postup_PPP_ALLOWED,Postup_POSS_PCT_ALLOWED,Spotup_PPP,Spotup_POSS_PCT,Spotup_PPP_ALLOWED,Spotup_POSS_PCT_ALLOWED,Handoff_PPP,Handoff_POSS_PCT,Handoff_PPP_ALLOWED,Handoff_POSS_PCT_ALLOWED,Cut_PPP,Cut_POSS_PCT,Cut_PPP_ALLOWED,Cut_POSS_PCT_ALLOWED,OffScreen_PPP,OffScreen_POSS_PCT,OffScreen_PPP_ALLOWED,OffScreen_POSS_PCT_ALLOWED,OffRebound_PPP,OffRebound_POSS_PCT,OffRebound_PPP_ALLOWED,OffRebound_POSS_PCT_ALLOWED,Isolation_PPP,Isolation_POSS_PCT,Isolation_PPP_ALLOWED,Isolation_POSS_PCT_ALLOWED,TOTAL_POSS_PCT,TOTAL_POSS_PCT_ALLOWED,NET_POINTS/ATTEMPT,IMPLIED_NET_POINTS/ATTEMPT,TOTAL_IMPLIED_POSS_PCT
0,0022000001,201142,BKN,2020-12-22,GSW,,1,1496.0,22.0,7.0,16.0,1.0,2.0,7.0,7.0,4.0,0.0,6.0,6.0,0.500,1.0,1.0,0.033,4.0,0.114,3.0,0.158,13.0,3.0,1.0,1.0,3.0,26.0,0.282,116.47,60.0,19.0,22.0,7.0,7.0,16.0,16.0,1.0,1.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0022000001,2020-21,Regular Season,BKN,GSW,GSW,None,None,19.0,0.316667,NaN,1.157895,NaN,NaN,NaN,1.129,0.260406,NaN,NaN,0.957,0.193170,NaN,NaN,1.000,0.050160,NaN,NaN,2.000,0.017076,NaN,NaN,0.905,0.176094,NaN,NaN,0.667,0.025614,NaN,NaN,1.143,0.058698,NaN,NaN,0.5,0.084312,NaN,NaN,1.714,0.058698,NaN,NaN,0.778,0.075774,0,0.937,0,0,0
1,0022000001,201145,BKN,2020-12-22,GSW,,0,1037.0,7.0,3.0,4.0,1.0,2.0,0.0,0.0,2.0,0.0,4.0,4.0,1.000,1.0,0.0,0.000,0.0,0.000,1.0,0.091,16.7,1.0,0.0,1.0,3.0,6.0,0.116,111.09,39.0,4.0,7.0,0.0,0.0,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0022000001,2020-21,Regular Season,BKN,GSW,GSW,None,None,4.0,0.102564,NaN,1.750000,NaN,NaN,NaN,1.129,0.260406,NaN,NaN,0.957,0.193170,NaN,NaN,1.000,0.050160,NaN,NaN,2.000,0.017076,NaN,NaN,0.905,0.176094,NaN,NaN,0.667,0.025614,NaN,NaN,1.143,0.058698,NaN,NaN,0.5,0.084312,NaN,NaN,1.714,0.058698,NaN,NaN,0.778,0.075774,0,0.937,0,0,0
2,0022000001,201599,BKN,2020-12-22,GSW,,1,1023.0,4.0,2.0,3.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,4.0,1.000,0.0,2.0,0.105,9.0,0.375,1.0,0.063,12.5,1.0,1.0,3.0,2.0,21.0,0.143,112.61,40.0,4.0,4.0,2.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0022000001,2020-21,Regular Season,BKN,GSW,GSW,None,None,4.0,0.100000,NaN,1.000000,NaN,NaN,NaN,1.129,0.260406,NaN,NaN,0.957,0.193170,NaN,NaN,1.000,0.050160,NaN,NaN,2.000,0.017076,NaN,NaN,0.905,0.176094,NaN,NaN,0.667,0.025614,NaN,NaN,1.143,0.058698,NaN,NaN,0.5,0.084312,NaN,NaN,1.714,0.058698,NaN,NaN,0.778,0.075774,0,0.937,0,0,0
3,0022000001,201939,GSW,2020-12-22,BKN,,1,1819.0,20.0,7.0,21.0,2.0,10.0,4.0,4.0,2.0,0.0,2.0,6.0,0.400,0.5,3.0,0.073,1.0,0.029,10.0,0.556,27.8,2.0,0.0,3.0,1.0,-23.0,0.313,117.16,74.0,23.0,20.0,4.0,4.0,21.0,16.0,0.0,0.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0022000001,2020-21,Regular Season,BKN,GSW,GSW,None,None,23.0,0.310811,NaN,0.869565,NaN,NaN,NaN,1.200,0.123789,0.909,1.0,0.565,0.190527,NaN,NaN,2.143,0.058127,NaN,NaN,0.000,0.024758,NaN,NaN,0.625,0.264801,NaN,NaN,0.600,0.040904,NaN,NaN,0.786,0.116254,NaN,NaN,0.8,0.040904,NaN,NaN,0.667,0.099031,NaN,NaN,1.000,0.040904,0,0.929,0,0,0
4,0022000001,202681,BKN,2020-12-22,GSW,,1,1518.0,26.0,10.0,16.0,4.0,7.0,2.0,2.0,3.0,5.0,3.0,10.0,0.333,0.5,1.0,0.033,3.0,0.086,4.0,0.222,18

In [8]:
for play_type in PLAY_TYPES:
    boxscores = feature_creation.expanding_weighted_mean(
        df=boxscores, group_col_names=['SEASON', 'TEAM', 'PLAYERID'],
        col_name='{}_PPP_ALLOWED'.format(play_type), weight_col_name='ATTEMPTS',
        new_col_name='AVG_{}_PPP_ALLOWED_PLAYED_AGAINST'.format(play_type)
    )
    boxscores = feature_creation.expanding_weighted_mean(
        df=boxscores, group_col_names=['SEASON', 'TEAM', 'PLAYERID'],
        col_name='{}_POSS_PCT_ALLOWED'.format(play_type), weight_col_name='POSS',
        new_col_name='AVG_{}_POSS_PCT_ALLOWED_PLAYED_AGAINST'.format(play_type)
    )

    try:
        boxscores['PPP_ADJ'] = boxscores.apply(
            lambda row: row['{}_PPP_ALLOWED'.format(play_type)]/row['AVG_{}_PPP_ALLOWED_PLAYED_AGAINST'.format(play_type)] \
                if (not np.isnan(row['{}_PPP_ALLOWED'.format(play_type)]) and \
                    not np.isnan(row['AVG_{}_PPP_ALLOWED_PLAYED_AGAINST'.format(play_type)])) \
                    else 1,
            axis = 1
            )
    except:
        display(boxscores.loc[boxscores['AVG_{}_PPP_ALLOWED_PLAYED_AGAINST'.format(play_type)] == 0])
        print(1/0)

    boxscores['POSS_PCT_ADJ'] = boxscores.apply(
        lambda row: row['{}_POSS_PCT_ALLOWED'.format(play_type)]/row['AVG_{}_POSS_PCT_ALLOWED_PLAYED_AGAINST'.format(play_type)] \
            if (not np.isnan(row['{}_POSS_PCT_ALLOWED'.format(play_type)]) and \
                not np.isnan(row['AVG_{}_POSS_PCT_ALLOWED_PLAYED_AGAINST'.format(play_type)])) \
                else 1,
        axis = 1
        )

    boxscores['NET_POINTS/ATTEMPT'] += \
        boxscores['{}_PPP'.format(play_type)].fillna(0) * boxscores['{}_POSS_PCT'.format(play_type)].fillna(0)
    boxscores['TOTAL_POSS_PCT'] += boxscores['{}_POSS_PCT'.format(play_type)].fillna(0)

    boxscores['IMPLIED_NET_POINTS/ATTEMPT'] += (boxscores['{}_PPP'.format(play_type)].fillna(0) * boxscores['PPP_ADJ']) * \
        (boxscores['{}_POSS_PCT'.format(play_type)].fillna(0) * boxscores['POSS_PCT_ADJ'])
    boxscores['TOTAL_IMPLIED_POSS_PCT'] += boxscores['{}_POSS_PCT'.format(play_type)].fillna(0) * boxscores['POSS_PCT_ADJ']

boxscores['POINTS/ATTEMPT_DEF_ADJ'] = boxscores.apply(
    lambda row: (row['IMPLIED_NET_POINTS/ATTEMPT']/row['TOTAL_IMPLIED_POSS_PCT']) - \
        (row['NET_POINTS/ATTEMPT']/row['TOTAL_POSS_PCT']) \
            if (row['TOTAL_POSS_PCT'] > 0 and row['TOTAL_IMPLIED_POSS_PCT'] > 0) else 0,
    axis = 1
)

,GAMEID,PLAYERID,TEAM,DATE,OPP_TEAM,COMMENT,START,SECONDSPLAYED,PTS,FGM,FGA,FG3M,FG3A,FTM,FTA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,PCT_AST_2PM,PCT_AST_3PM,OREB,OREB_PCT,DREB,DREB_PCT,AST,AST_PCT,AST_RATIO,STL,BLK,TOV,PF,PLUSMINUS,USG_PCT,PACE,POSS,TOTAL_ATTEMPTS,TOTAL_PTS,TOTAL_FTA,TOTAL_FTM,SHOT_ATTEMPTS,SHOT_PTS,SHOT_FTA,SHOT_FTM,SFOUL_ATTEMPTS,SFOUL_PTS,SFOUL_FTA,SFOUL_FTM,PFOUL_ATTEMPTS,PFOUL_PTS,PFOUL_FTA,PFOUL_FTM,TFOUL_ATTEMPTS,TFOUL_PTS,TFOUL_FTA,TFOUL_FTM,ID,SEASON,SEASONTYPE,HTM,VTM,W,NAME,POSITION,ATTEMPTS,ATTEMPTS/POSSESSION,AVG_ATTEMPTS/POSSESSION,POINTS/ATTEMPT,AVG_POINTS/ATTEMPT,Transition_PPP,Transition_POSS_PCT,Transition_PPP_ALLOWED,Transition_POSS_PCT_ALLOWED,PRBallHandler_PPP,PRBallHandler_POSS_PCT,PRBallHandler_PPP_ALLOWED,PRBallHandler_POSS_PCT_ALLOWED,PRRollman_PPP,PRRollman_POSS_PCT,PRRollman_PPP_ALLOWED,PRRollman_POSS_PCT_ALLOWED,Postup_PPP,Postup_POSS_PCT,Postup_PPP_ALLOWED,Postup_POSS_PCT_ALLOWED,Spotup_PPP,Spotup_POSS_PCT,Spotup_PPP_ALLOWED,Spotup_POSS_PCT_ALLOWED,Handoff_PPP,Handoff_POSS_PCT,Handoff_PPP_ALLOWED,Handoff_POSS_PCT_ALLOWED,Cut_PPP,Cut_POSS_PCT,Cut_PPP_ALLOWED,Cut_POSS_PCT_ALLOWED,OffScreen_PPP,OffScreen_POSS_PCT,OffScreen_PPP_ALLOWED,OffScreen_POSS_PCT_ALLOWED,OffRebound_PPP,OffRebound_POSS_PCT,OffRebound_PPP_ALLOWED,OffRebound_POSS_PCT_ALLOWED,Isolation_PPP,Isolation_POSS_PCT,Isolation_PPP_ALLOWED,Isolation_POSS_PCT_ALLOWED,TOTAL_POSS_PCT,TOTAL_POSS_PCT_ALLOWED,NET_POINTS/ATTEMPT,IMPLIED_NET_POINTS/ATTEMPT,TOTAL_IMPLIED_POSS_PCT,AVG_Transition_PPP_ALLOWED_PLAYED_AGAINST,AVG_Transition_POSS_PCT_ALLOWED_PLAYED_AGAINST,PPP_ADJ,POSS_PCT_ADJ,AVG_PRBallHandler_PPP_ALLOWED_PLAYED_AGAINST,AVG_PRBallHandler_POSS_PCT_ALLOWED_PLAYED_AGAINST,AVG_PRRollman_PPP_ALLOWED_PLAYED_AGAINST,AVG_PRRollman_POSS_PCT_ALLOWED_PLAYED_AGAINST,AVG_Postup_PPP_ALLOWED_PLAYED_AGAINST,AVG_Postup_POSS_PCT_ALLOWED_PLAYED_AGAINST
452,1e+24,1629672,GSW,2020-12-25,MIL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-21,NaN,NaN,NaN,NaN,Eric Paschall,Forward,NaN,NaN,0.142857,NaN,0.857143,NaN,NaN,1.154,0.122363,NaN,NaN,0.741,0.254219,NaN,NaN,1.571,0.066456,NaN,NaN,1.667,0.028481,NaN,NaN,1.208,0.225738,NaN,NaN,1.167,0.056962,NaN,NaN,2.000,0.037975,NaN,NaN,0.75,0.037975,NaN,NaN,1.143,0.066456,NaN,NaN,0.818,0.103376,0.0,0.948,0.000,0.000000,0.000000,1.2,0.123789,0.733084,1.143284,0.565,0.190527,2.143,0.058127,0.0,0.024758
453,1e+24,1626162,GSW,2020-12-25,MIL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-21,NaN,NaN,NaN,NaN,Kelly Oubre Jr.,Forward-Guard,NaN,NaN,0.222222,NaN,0.428571,NaN,NaN,1.154,0.122363,NaN,NaN,0.741,0.254219,NaN,NaN,1.571,0.066456,NaN,NaN,1.667,0.028481,NaN,NaN,1.208,0.225738,NaN,NaN,1.167,0.056962,NaN,NaN,2.000,0.037975,NaN,NaN,0.75,0.037975,NaN,NaN,1.143,0.066456,NaN,NaN,0.818,0.103376,0.0,0.948,0.000,0.000000,0.000000,1.2,0.123789,0.733084,1.143284,0.565,0.190527,2.143,0.058127,0.0,0.024758
454,1e+24,203952,GSW,2020-12-25,MIL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-21,NaN,NaN,NaN,NaN,Andrew Wiggins,Forward,NaN,NaN,0.250000,NaN,0.722222,NaN,NaN,1.154,0.122363,NaN,NaN,0.741,0.254219,NaN,NaN,1.571,0.066456,NaN,NaN,1.667,0.028481,NaN,NaN,1.208,0.225738,NaN,NaN,1.167,0.056962,NaN,NaN,2.000,0.037975,NaN,NaN,0.75,0.037975,NaN,NaN,1.143,0.066456,NaN,NaN,0.818,0.103376,0.0,0.948,0.000,0.000000,0.000000,1.2,0.123789,0.733084,1.143284,0.565,0.190527,2.143,0.058127,0.0,0.024758
455,1e+24,201939,GSW,2020-12-25,MIL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

ZeroDivisionError: division by zero